In [1]:
import numpy as np
import math
from functools import reduce
import time
import sys
import os
import numpy as np
from matplotlib import pyplot as plt
path_project = "\\".join(os.getcwd().split("\\")[:-1])
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, path_project)
from pathlib import Path
import nidaqmx

def lcm(a, b):
    """Calculate the least common multiple of two numbers."""
    return abs(a * b) // math.gcd(a, b)


def lcm_of_list(numbers):
    """Find the LCM of a list of numbers."""
    return reduce(lcm, numbers)

def seqtime(seq_tb):
    return np.sum([pulse[-1] for pulse in seq_tb])

# some constants
Hz = 1e-9 # GHz
kHz = 1e-6 # GHz
MHz = 1e-3 # GHz
pi = np.pi

In [2]:
import nidaqmx
from nidaqmx.constants import TerminalConfiguration, VoltageUnits, Edge, AcquisitionType, READ_ALL_AVAILABLE
from nidaqmx.stream_readers import AnalogSingleChannelReader

from hardware import config as hcf
from hardware.hardwaremanager import HardwareManager
from hardware.pulser.pulser import (
    OutputState,
    TriggerStart,
    TriggerRearm,
    HIGH,
    LOW,
    INF,
    REPEAT_INFINITELY
)
timebase = lcm_of_list(
    [hcf.VDISYN_timebase, hcf.SIDIG_timebase, hcf.PS_timebase, hcf.RSRF_timebase]
)

hw = HardwareManager()
# adds synthesizer, laser, and pulse generator 
hw.add_default_hardware()


Connect to Pulse Streamer via JSON-RPC.
IP / Hostname: 169.254.8.2
Pulse Streamer 8/2 firmware: v1.7.2
Client software: v1.7.0
VDI Sythesizer Serail Port Open


In [17]:
paraset = dict(
    ac_freq = 5, # MHz
    tau_dd = 30,
    t_pi_mwa = 70,
    t_pi_mwb = 70,
     rate_refresh=10.0,
    # --------------------
    laser_current=30.0,  # percentage
    mw_freq=392.83924,  # GHz
    mw_powervolt=5.0,  # voltage 0.0 to 5.0
    mw_phasevolt=7.72,  # voltage 0.0 to 5.0
    amp_input=1000,  # input amplitude for digitizer
    bgextend_size=256,  # TODO: why 256? is it a fixed number?
    # -------------------
    init_nslaser=50,  # [ns]
    init_isc=150,
    init_repeat=40,
    init_wait=1000.0,
    # t_pi_mwa=90,
    # t_pi_mwb=90,
    read_wait=300.0,
    read_laser=900.0,
    tau_begin=0.0,
    tau_end=10000,
    tau_step=100.0,  # [ns]
    n_pi=2,
)

In [18]:
# def sequence_ts(tau):

#         # handling for sin wave 
#         ac_freq=  paraset["ac_freq"]
        
#         tau_dd = int( paraset["tau_dd"])
#         t_pi_mwa = int( paraset["t_pi_mwa"])
#         t_pi_mwb = int( paraset["t_pi_mwb"])

#         t_pio2_mwa = int(t_pi_mwa / 2)
#         t_pio2_mwb = int(t_pi_mwb / 2)

#         ac_duration = int( t_pio2_mwa/2 + 2*(tau_dd)+t_pi_mwa+t_pio2_mwb/2) #ns
#         ac_samplerate=1e9 # 1GS/s
#         samples_per_cycle= int(ac_samplerate/(ac_freq*1e6)) 
#         total_samples= int (ac_duration)

#         # Generate time steps and sine values
#         t = np.arange(total_samples)
#         cos_wave = np.cos(2 * np.pi * ac_freq * t / ac_samplerate)

#         # Convert to pulse pattern [(duration, value)]
#         pulse_pattern = [(1, float(val)) for val in cos_wave]
#         analog_pio2o2_wait=[((int(t_pio2_mwa/2)),0.0)]
#         analog_pause= [(tau,0.0)]

#         sq_dd = [
#             (["mwA"], t_pio2_mwa),
#             ([], tau_dd),
#             (["mwA"], t_pi_mwa),
#             ([], tau_dd),
#             (["mwB"], t_pio2_mwb),
#         ]

#         seq_fe = [([], tau)]
#         seq_ts = sq_dd + seq_fe + sq_dd
#         seq_analog =analog_pio2o2_wait+pulse_pattern+analog_pause+2*analog_pio2o2_wait+pulse_pattern
#         tau_ext = 0
#         len_dig=get_sequence_duration(seq_ts)
#         len_analogue=get_analog_duration(seq_analog)
#         seq_analog+=[(len_dig-len_analogue,0.0)]
        
#         return seq_ts, tau_ext ,seq_analog



In [19]:
# seq_ts_all, tau_ext_all, seq_analog_all = [],[],[]
# for i in range (10):
    
#     seq_ts, tau_ext, seq_analog = sequence_ts(10*i)
#     seq_ts_all+=seq_ts
#     # tau_ext_all+=tau_ext
#     seq_analog_all+=seq_analog
#     # print(get_sequence_duration(seq_ts_all))
#     # print(get_analog_duration(seq_analog_all))
#     print(i)
# # 2. Set the digital and analog channels
# hw.pg.resetSeq()  # clears old sequences
# hw.pg.setSequence(seq_ts_all)  # set digital sequence
# hw.pg.setAnalog("Bz", seq_analog_all)  # set analog output on ch8

# # 3. Set trigger and stream
# hw.pg.setTrigger(start=TriggerStart.SOFTWARE, rearm=TriggerRearm.MANUAL)
# # hw.pg.stream()


In [20]:
# hw.pg.plotSeq(plot_all=False)


In [21]:
def get_sequence_duration(seq):
    total_duration = 0
    for step in seq:
        if not isinstance(step, tuple) or len(step) != 2:
            raise ValueError(f"Invalid sequence step: {step}")
        _, duration = step
        if not isinstance(duration, (int, float)):
            raise ValueError(f"Duration must be int or float, got {type(duration)}")
        total_duration += int(duration)
    return total_duration


def get_analog_duration(pattern):
    total_duration = 0
    for step in pattern:
        if not isinstance(step, tuple) or len(step) != 2:
            raise ValueError(f"Invalid analog step: {step}")
        duration, value = step
        if not isinstance(duration, (int, float)):
            raise ValueError(f"Duration must be int or float, got {type(duration)}")
        total_duration += int(duration)
    return total_duration


In [40]:
def seq_init(init_nslaser: int, init_isc: int, init_wait: int, init_repeat: int):
    return [(["laser"], init_nslaser), ([], init_isc)] * init_repeat + [([], init_wait)]


def seq_read(read_wait: int, read_laser: int):
    return [([], read_wait), (["laser", "sdtrig"], read_laser)]

def _sequence_ts( tau):
    ac_freq = paraset["ac_freq"]

    tau_dd = int(paraset["tau_dd"])
    tau_dd_half = int(tau_dd / 2)
    t_pi_mwa = int(paraset["t_pi_mwa"])
    t_pi_mwb = int(paraset["t_pi_mwb"])

    t_pio2_mwa = int(t_pi_mwa / 2)
    t_pio2_mwb = int(t_pi_mwb / 2)
    # dur_dd = t_pio2_mwa / 2 + 2 * (tau_dd) + t_pi_mwa + t_pio2_mwb / 2 # for hahn echo
    n_XY8 = 1
    dur_dd = (
        t_pio2_mwa / 2
        + (tau_dd * 6
        + tau_dd_half * 2
        + t_pi_mwb * 4
        + t_pi_mwa * 4)*n_XY8
        + t_pio2_mwb / 2
    )  # for XY8
    ac_duration = int(dur_dd + t_pio2_mwb + tau + t_pio2_mwa + dur_dd)  # ns
    ac_samplerate = paraset["ac_freq"] / 1e3 * 20.0  # 20 pt per period

    # Generate time steps and sine values
    dt = int(1 / ac_samplerate)
    ttt = np.arange(0.0, ac_duration, int(1 / ac_samplerate))  # ns
    # cos_wave = np.sin(2 * np.pi * ac_freq * ttt / 1e3)  # for hahn echo
    cos_wave = np.cos(2 * np.pi * ac_freq * ttt / 1e3)  # for XY8

    # Convert to pulse pattern [(duration, value)]
    pulse_pattern = [(dt, float(val)) for val in cos_wave]
    analog_pio2o2_wait = [((int(t_pio2_mwa )), 0.0)]
    seq_ts = []
    seq_ts += [(["mwA"], t_pio2_mwa)]

    XY8_seq = []
    XY8_seq += [([], tau_dd_half), (["mwA"], t_pi_mwa), ([], tau_dd)]  # X
    XY8_seq += [(["mwB"], t_pi_mwb), ([], tau_dd)]  # Y
    XY8_seq += [(["mwA"], t_pi_mwa), ([], tau_dd)]  # X
    XY8_seq += [(["mwB"], t_pi_mwb), ([], tau_dd)]  # Y

    XY8_seq += [(["mwB"], t_pi_mwb), ([], tau_dd)]  # Y
    XY8_seq += [(["mwA"], t_pi_mwa), ([], tau_dd)]  # X
    XY8_seq += [(["mwB"], t_pi_mwb), ([], tau_dd)]  # Y
    XY8_seq += [(["mwA"], t_pi_mwa), ([], tau_dd_half)]  # X

    XY8_seq_n = XY8_seq * n_XY8
    seq_ts += XY8_seq_n

    seq_ts += [(["mwB"], t_pio2_mwb)]
    sq_dd = seq_ts
    # sq_dd = [
    #     (["mwA"], t_pio2_mwa),
    #     ([], tau_dd),
    #     (["mwA"], t_pi_mwa),
    #     ([], tau_dd),
    #     (["mwB"], t_pio2_mwb),
    # ]

    seq_fe = [([], tau)]
    seq_ts = sq_dd + seq_fe + sq_dd
    seq_analog = analog_pio2o2_wait + pulse_pattern + analog_pio2o2_wait
    tau_ext = 0
    len_dig = get_sequence_duration(seq_ts)
    len_analogue = get_analog_duration(seq_analog)
    seq_analog += [(len_dig - len_analogue, 0.0)]

    return seq_ts, tau_ext, seq_analog

def sequence():


    # dark ref + dark ref + tau sweep sequence
    sq_init = seq_init(
            paraset["init_nslaser"],
            paraset["init_isc"],
            paraset["init_wait"],
            paraset["init_repeat"],
    )
    sq_init_analog = [(get_sequence_duration(sq_init), 0.0)]
    sq_read = seq_read( paraset["read_wait"],  paraset["read_laser"])
    sq_read_analog = [(get_sequence_duration(sq_read), 0.0)]
    sq_exp = []
    seq_analog_exp = []
    # start with a bright and dark reference
    sq_dark = sq_init + [(["mwA"],  paraset["t_pi_mwa"])] + sq_read
    sq_bright = sq_init + [([],  paraset["t_pi_mwa"])] + sq_read
    sq_exp += sq_dark + sq_bright
    seq_analog_exp += [(get_sequence_duration(sq_dark),0.0)] + [(get_sequence_duration(sq_bright),0.0)]
    # add tau sweep sequence
    tau_begin =  paraset["tau_begin"]
    tau_end =  paraset["tau_end"]
    tau_step =  paraset["tau_step"]
    tauarray = np.arange(tau_begin, tau_end + tau_step, tau_step)
    tauaprime = np.arange(tau_begin, tau_end + tau_step, tau_step)

    for ii, tau in enumerate(tauarray):
        sq_ts, tau_ext, seq_analog =  _sequence_ts(tau)
        tauaprime[ii] = tau + tau_ext

        sq_exp += sq_init + sq_ts + [([],  paraset["t_pi_mwa"])] + sq_read
        seq_analog_exp += (
            sq_init_analog
            + seq_analog
            + [( paraset["t_pi_mwa"], 0.0)]
            + sq_read_analog
        )

        sq_exp += sq_init + sq_ts + [(["mwA"],  paraset["t_pi_mwa"])] + sq_read
        seq_analog_exp += (
            sq_init_analog
            + seq_analog
            + [( paraset["t_pi_mwa"], 0.0)]
            + sq_read_analog
        )
    return sq_exp, tauaprime, seq_analog_exp


In [41]:
seq_exp, tauaprime, seq_analog_exp = sequence()

In [42]:
ac_samplerate = paraset["ac_freq"] / 1e6 * 20.0  # 20 pt per period

In [43]:
paraset["ac_freq"] 

5

In [44]:
ac_samplerate

0.0001

In [45]:

hw.pg.resetSeq()  # clears old sequences
hw.pg.setSequence(seq_exp)  # set digital sequence
hw.pg.setAnalog("Bz", seq_analog_exp)  # set analog output on ch8

# 3. Set trigger and stream
hw.pg.setTrigger(start=TriggerStart.SOFTWARE, rearm=TriggerRearm.MANUAL)
# hw.pg.stream()


Setting Analog channel 0


In [46]:
ac_freq = paraset["ac_freq"]
tau = 100.0
tau_dd = int(paraset["tau_dd"])
t_pi_mwa = int(paraset["t_pi_mwa"])
t_pi_mwb = int(paraset["t_pi_mwb"])

t_pio2_mwa = int(t_pi_mwa / 2)
t_pio2_mwb = int(t_pi_mwb / 2)
dur_dd = t_pio2_mwa / 2 + 2 * (tau_dd) + t_pi_mwa + t_pio2_mwb / 2
ac_duration = int(dur_dd + tau + dur_dd - t_pio2_mwa / 2 * 2)  # ns

In [47]:
ac_duration

395

In [48]:
hw.pg.plotSeq(plot_all=False)
